In [1]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import jax
import jax.numpy as jnp
import numpy as np

import jaxmao
from jaxmao.layers import Conv2D, SimpleDense, Dense, BatchNorm, ReLU, Flatten, StableSoftmax, BatchNorm2D, DepthwiseConv2D, Activation
from jaxmao.modules import Module
from jaxmao.optimizers import GradientDescent
from jaxmao.losses import CategoricalCrossEntropy
from jaxmao.metrics import Accuracy, Precision, Recall

print('jax.devices() :', jax.devices())

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
print('tf.config.list_physical_devices(): ', tf.config.list_physical_devices())

seed = 42
key = jax.random.PRNGKey(seed)

tf.keras.backend.set_floatx('float32')

I0000 00:00:1698063410.970257  134056 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
2023-10-23 19:16:50.990324: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:276] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-23 19:16:51.298098: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 19:16:51.298187: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 19:16:51.298222: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


jax.devices() : [CpuDevice(id=0)]


2023-10-23 19:16:52.371070: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


tf.config.list_physical_devices():  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2023-10-23 19:16:53.265906: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
def close_enough(A, B, eps=1e-5):
    return np.less_equal(np.abs(A - B), eps)

# jax.grad vs tf.GradientTape

In [3]:
# Define the function f(x) = x^2
def jax_function(x):
    return jax.lax.pow(x, 2)

def tf_function(x):
    return tf.pow(x, 2)

input = [2.0, 5.4]

# Compute the gradient using JAX
jax_grad = jax.grad(jax_function)
x_jax = jnp.array(input)
grad_jax = jax.vmap(jax_grad)(x_jax)

# Compute the gradient using TensorFlow
x_tf = tf.Variable(input, dtype=tf.float32)
with tf.GradientTape() as tape:
    tape.watch(x_tf)
    y_tf = tf_function(x_tf)
grad_tf = tape.gradient(y_tf, x_tf).numpy()

# Compare
print("JAX Gradient:", grad_jax)
print("TensorFlow Gradient:", grad_tf)
print("Are they close enough?", np.isclose(grad_jax, grad_tf, atol=1e-6))

JAX Gradient: [ 4.  10.8]
TensorFlow Gradient: [ 4.  10.8]
Are they close enough? [ True  True]


# simple module

In [4]:
X_train = np.random.normal(2, 4, (200, 8, 8, 1)).astype('float32')
y_train = np.random.randint(0, 10, (200,)).astype('float32')
y_train_enc = np.array(
    jax.nn.one_hot(y_train, num_classes=10)
)

In [5]:
class DenseMNISTClasifier(Module):
    def __init__(self):
        super().__init__()
        self.add('bn1', jaxmao.layers.BatchNorm2D(1, momentum=0.99, eps=1e-5))
        self.add('conv1', jaxmao.layers.Conv2D(1, 4, (3, 3), (1,1), 'relu', weights_initializer=jaxmao.initializers.GlorotNormal()))
        self.add('flatten', jaxmao.layers.Flatten())
        self.add('dense1', jaxmao.layers.Dense(8*8*4, 128, 'relu', weights_initializer=jaxmao.initializers.GlorotNormal()))
        self.add('dense2', jaxmao.layers.Dense(128, 32, 'relu', weights_initializer=jaxmao.initializers.GlorotNormal()))
        self.add('dense3', jaxmao.layers.Dense(32, 10, 'softmax', weights_initializer=jaxmao.initializers.GlorotNormal()))
    
    def forward(self, params, x, state):
        x, state = self.apply(params, x, 'bn1', state)
        x, state = self.apply(params, x, 'conv1', state)
        x, state = self.apply(params, x, 'flatten', state)
        x, state = self.apply(params, x, 'dense1', state)
        x, state = self.apply(params, x, 'dense2', state)
        x, state = self.apply(params, x, 'dense3', state)
        return x, state
    
jaxmao_model = DenseMNISTClasifier()
jaxmao_model.init_params(key)
summary = jaxmao_model.summarize(input_shape=(4, 8, 8, 1))

print('\n\n')
# Initialize the Sequential model
keras_model = keras.Sequential(name='keras_denseMNIST')

# Add layers to the keras_model
keras_model.add(keras.layers.BatchNormalization(momentum=0.99, input_shape=(8, 8, 1), epsilon=1e-5))
keras_model.add(keras.layers.Conv2D(filters=4, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
keras_model.add(keras.layers.Flatten())
keras_model.add(keras.layers.Dense(units=128, activation='relu'))
keras_model.add(keras.layers.Dense(units=32, activation='relu'))
keras_model.add(keras.layers.Dense(units=10, activation='softmax'))

# Summary of the keras_model to show the architecture
keras_model.summary()

(3, 3, 1, 4)         45        
(256, 128)           384       
(128, 32)            160       
(32, 10)             42        
layer                output shape         #'s params           #'s states          
bn1                  (4, 8, 8, 1)         2                    0                   
conv1                (4, 8, 8, 4)         40                   0                   
flatten              (4, 256)             0                    0                   
dense1               (4, 128)             32896                0                   
dense2               (4, 32)              4128                 0                   
dense3               (4, 10)              330                  0                   

total parameters: 37396



Model: "keras_denseMNIST"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (Batch  (None, 8, 8, 1)           4         
 Normalization)                 

In [6]:
for index, (jaxmao_layer, keras_layer) in enumerate(zip(jaxmao_model.layers.values(), keras_model.layers)):
    if isinstance(jaxmao_layer, jaxmao.layers.BatchNorm):
        keras_model.layers[index].set_weights(([np.array(jaxmao_layer.params['gamma']), 
                                                np.array(jaxmao_layer.params['beta']), 
                                                np.array(jaxmao_layer.state['running_mean']), 
                                                np.array(jaxmao_layer.state['running_var'])
                                                ]))
    elif isinstance(jaxmao_layer, jaxmao.layers.Conv2D):
        keras_model.layers[index].set_weights([np.array(value) for value in jaxmao_layer.params['conv2d/simple_conv2d'].values()])
        # TODO if the Conv2D has batchnorm.
    elif isinstance(jaxmao_layer, jaxmao.layers.Flatten):
        pass
    elif isinstance(jaxmao_layer, jaxmao.layers.Dense):
        keras_model.layers[index].set_weights([np.array(value) for value in jaxmao_layer.params['dense/simple_dense'].values()])
        # TODO if the Dense has batchnorm.

In [7]:
close_prediction1 = close_enough(keras_model(X_train).numpy(), jaxmao_model(jaxmao_model.params, X_train), 1e-7)
np.prod(close_prediction1.shape), close_prediction1.sum()

(2000, 1999)

### loss grad

In [8]:
loss_fn = jaxmao.losses.CategoricalCrossEntropy('mean_over_batch_size')
optimizer = jaxmao.optimizers.GradientDescent(lr=0.01, params=jaxmao_model.params)

def _loss_fnx(pure_forward, params, x, y, state):
    y_pred, new_state = pure_forward(params, x, state)
    loss = loss_fn(y_pred, y)
    return loss, new_state

loss_and_grad = jax.value_and_grad(_loss_fnx, argnums=1, has_aux=True)
# (loss, new_state), gradients = jax.block_until_ready(
#     loss_and_grad(jaxmao_model.pure_forward, jaxmao_model.params, X_train, y_train_enc, jaxmao_model.state)
# )
loss_fn(jaxmao_model(jaxmao_model.params, X_train), y_train_enc)

Array(2.952032, dtype=float32)

In [9]:
loss_fn(np.array(keras_model(X_train)), y_train_enc) # keras model, jaxmao loss_fn

Array(2.952032, dtype=float32)

In [10]:
keras.losses.CategoricalCrossentropy(reduction='sum_over_batch_size')(y_train_enc, keras_model(X_train)).numpy()

2.9520326

In [11]:
keras.losses.CategoricalCrossentropy(reduction='sum_over_batch_size')(y_train_enc, jaxmao_model(jaxmao_model.params, X_train)).numpy()

2.9520326

#### JaxMao SGD fit

In [12]:
from sklearn.metrics import accuracy_score

EPOCHS = 20
BATCH_SIZE = 10
NUM_BATCHES = len(X_train) // BATCH_SIZE

for epoch in range(EPOCHS):
    losses = 0.0
    # we can shuffle training set here
    for num_batch in range(NUM_BATCHES):
        i_index = num_batch*BATCH_SIZE
        e_index = (num_batch+1)*BATCH_SIZE
        batch_x = X_train[i_index: e_index]
        batch_y = y_train_enc[i_index: e_index]
        
        (loss, new_state), gradients = loss_and_grad(jaxmao_model.pure_forward, jaxmao_model.params, 
                                                     batch_x, batch_y, jaxmao_model.state)
        jaxmao_model.params, optimizer.state = optimizer(jaxmao_model.params, gradients, optimizer.state)
        jaxmao_model.update_state(new_state)
        
        losses += loss

    pred = jaxmao_model(jaxmao_model.params, X_train)    
    print('epoch {}: loss: {}, accuracy: {}'.format(epoch+1, losses/NUM_BATCHES, accuracy_score(y_train, pred.argmax(axis=1))))

epoch 1: loss: 2.642361879348755, accuracy: 0.155
epoch 2: loss: 2.286381721496582, accuracy: 0.225
epoch 3: loss: 2.1648850440979004, accuracy: 0.31
epoch 4: loss: 2.068301200866699, accuracy: 0.355
epoch 5: loss: 1.961280107498169, accuracy: 0.42
epoch 6: loss: 1.839827537536621, accuracy: 0.48
epoch 7: loss: 1.701788306236267, accuracy: 0.555
epoch 8: loss: 1.5397034883499146, accuracy: 0.615
epoch 9: loss: 1.363107442855835, accuracy: 0.695
epoch 10: loss: 1.1714884042739868, accuracy: 0.79
epoch 11: loss: 0.9792134165763855, accuracy: 0.825
epoch 12: loss: 0.7859901189804077, accuracy: 0.875
epoch 13: loss: 0.6146777272224426, accuracy: 0.915
epoch 14: loss: 0.46759486198425293, accuracy: 0.93
epoch 15: loss: 0.3600350618362427, accuracy: 0.96
epoch 16: loss: 0.2795087695121765, accuracy: 0.975
epoch 17: loss: 0.21337072551250458, accuracy: 0.99
epoch 18: loss: 0.16679050028324127, accuracy: 0.99
epoch 19: loss: 0.13324476778507233, accuracy: 0.99
epoch 20: loss: 0.107745431363582

#### Keras SGD fit

In [13]:
keras_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01), 
                    loss=keras.losses.CategoricalCrossentropy(reduction='sum_over_batch_size'), 
                    metrics=['accuracy'])
history = keras_model.fit(X_train, y_train_enc, epochs=20, batch_size=10, shuffle=False)

Epoch 1/20


20/20 [==============================] - 1s 5ms/step - loss: 2.3482 - accuracy: 0.1000
Epoch 2/20
20/20 [==============================] - 0s 4ms/step - loss: 2.3129 - accuracy: 0.1100
Epoch 3/20
20/20 [==============================] - 0s 4ms/step - loss: 2.2858 - accuracy: 0.1400
Epoch 4/20
20/20 [==============================] - 0s 4ms/step - loss: 2.2628 - accuracy: 0.1500
Epoch 5/20
20/20 [==============================] - 0s 4ms/step - loss: 2.2422 - accuracy: 0.1550
Epoch 6/20
20/20 [==============================] - 0s 3ms/step - loss: 2.2220 - accuracy: 0.1800
Epoch 7/20
20/20 [==============================] - 0s 3ms/step - loss: 2.2016 - accuracy: 0.1900
Epoch 8/20
20/20 [==============================] - 0s 3ms/step - loss: 2.1803 - accuracy: 0.2100
Epoch 9/20
20/20 [==============================] - 0s 2ms/step - loss: 2.1576 - accuracy: 0.2200
Epoch 10/20
20/20 [==============================] - 0s 3ms/step - loss: 2.1333 - accuracy: 0.2350
Epoch 11/20
20/20 [===========